In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

# Liste der Ferienarten
ferienarten = ['winter', 'oster', 'pfingst', 'sommer', 'herbst', 'weihnachten']

# 2000 bis 2025
jahre = range(2000, 2026)

ferien_daten = []

def clean_date_str(s):
    return s.replace('+', '').replace('–', '-').strip()

def parse_date(date_str):
    try:
        return datetime.strptime(date_str.strip(), '%d.%m.%Y')
    except:
        return None

for jahr in jahre:
    for ferienart in ferienarten:
        url = f'https://www.schulferien.org/deutschland/ferien/{ferienart}/{jahr}/'
        response = requests.get(url)
        if response.status_code != 200:
            continue
        soup = BeautifulSoup(response.content, 'html.parser')
        tables = soup.find_all('table')

        for table in tables:
            ferientyp_raw = table.find("th", class_="sf_table_header_cell")
            if not ferientyp_raw:
                continue
            ferientyp = ferientyp_raw.get_text(strip=True).split()[0]

            rows = table.find("tbody").find_all("tr")
            for row in rows:
                if not row.find("td"):
                    continue

                bundesland_tag = row.find("span", class_="sf_table_index_row_value")
                datum_tag = row.find("span", class_="nowrap")

                if not (bundesland_tag and datum_tag):
                    continue

                bundesland = bundesland_tag.get_text(strip=True)
                date_range_raw = datum_tag.get_text(strip=True)

                # Pluszeichen und überflüssige Leerzeichen
                date_range = clean_date_str(date_range_raw)

                try:
                    # Pluszeichen und doppelten Whitespace
                    cleaned_range = clean_date_str(date_range)

                    # Datumsintervall: "01.01.2022 - 05.01.2022"
                    if ' - ' in cleaned_range:
                        start_str, end_str = [s.strip() for s in cleaned_range.split(' - ')]
                        start_date = parse_date(start_str)
                        end_date = parse_date(end_str)

                    # 2. Format: "01.01.2022 + 02.01.2022 +" oder "01.01.2022 02.01.2022"
                    elif len(cleaned_range.split()) >= 2:
                        parts = [p.strip(" +") for p in cleaned_range.split() if p.strip(" +")]
                        start_date = parse_date(parts[0])
                        end_date = parse_date(parts[1])

                    # 3. Nur ein Datum vorhanden (eintägige Ferien)
                    else:
                        start_date = end_date = parse_date(cleaned_range)

                    # Validierung
                    if not start_date or not end_date:
                        raise ValueError("Ungültiges Datum")

                    # In Einzeltage auflösen
                    current = start_date
                    while current <= end_date:
                        ferien_daten.append({
                            "Datum": current.strftime('%Y-%m-%d'),
                            "Bundesland": bundesland,
                            "Ferientyp": ferientyp
                        })
                        current += timedelta(days=1)

                except Exception as e:
                    print(f"[{jahr} - {ferienart}] Fehler beim Parsen: '{date_range_raw}' → {e}")

# Speichern
df = pd.DataFrame(ferien_daten)
df.to_csv('ferientage_2000_2025.csv', index=False, encoding='utf-8')

✅ CSV erfolgreich erstellt mit 29393 Einträgen.
